In [184]:
totalValuesIntransctions= []
Transactions = []
c={}
l={}
discarded_transactions={} 
supportCount  = {}


# Reading the txt file

In [192]:
def test():
    resultedFile= []
    totalRows = 0
    
    with open('../data.txt','r') as f: 
        lines = f.readlines()



    for line in lines: 
        if totalRows==0:
            totalRows=line
        else: 
            # updatedFile = line.replace('\t','')
            # updatedFile = updatedFile.replace(' ','')
            updatedFile = line
            totalValuesIntransctions.append(updatedFile[1])
            print(line[3:])
            updatedFile = updatedFile[2:]
            updatedFile=list(updatedFile.replace('\n',''))

            resultedFile.append(updatedFile)
        

    return resultedFile, totalRows
    


In [193]:
Transactions,totalRows=test()
print(Transactions)

	1 3 4

	2 3 5

	1 2 3 5

	2 5 

[['3', '\t', '1', ' ', '3', ' ', '4'], ['3', '\t', '2', ' ', '3', ' ', '5'], ['4', '\t', '1', ' ', '2', ' ', '3', ' ', '5'], ['2', '\t', '2', ' ', '5', ' ']]


# Minimum Threshold 

In [173]:
minimum_confidence_percent = 50
minimumSupport = (int(totalRows)*minimum_confidence_percent)/100
print(minimumSupport)

500.0


# Counting the items

In [174]:
from collections import Counter
totalRowsCounter=0
totalItems = Counter()
for line in Transactions:

    if totalRowsCounter==0:
        totalItems=(Counter(line))
    else:
        totalItems=(Counter(line)) + totalItems
    totalRowsCounter = totalRowsCounter+1
    

totalItems=dict(totalItems)
print('C1')
print(totalItems)
print(list(totalItems.values()))


C1
{'0': 1458, '5': 1814, '2': 1910, '3': 1917, '4': 2162, '1': 2058, '6': 1807, '8': 2103, '9': 1955, '7': 1855}
[1458, 1814, 1910, 1917, 2162, 2058, 1807, 2103, 1955, 1855]


In [175]:

initialSize = 1
totalSize =1
sortedOrder = sorted(totalItems.keys())
c.update({initialSize : [[f] for f in sorted(totalItems.keys())] })

discarded_transactions.update({initialSize:[]})
print(c)

{1: [['0'], ['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9']]}


In [176]:
def count_item(singleItemSet, Transactions):
    counter = 0
    for i in range(len(Transactions)):
        if set(singleItemSet).issubset(set(Transactions[i])):
            counter = counter+1
    return counter

In [177]:
def get_frequent(itemSet, initialTransactions, minimumSupport,discarded_transactions):
    tempL = []
    newDiscardedValue = []
    itemCount = []
    # First need to check if the itemset contains subset of items that was previously discarded
    for i in range(len(itemSet)):
        isDiscarded = False
        # if lenght of discarded items is not 0
        if len(discarded_transactions.keys()) > 0:
            # this loop checks if there is any value in the item that was discarded before 
            for k in discarded_transactions[len(discarded_transactions.keys())]:
                
                if set(k).issubset(set(itemSet[i])):
                    isDiscarded = True
                    break
        if isDiscarded == False:
            itemCounter = count_item(itemSet[i],Transactions)
            if itemCounter >= minimumSupport:
                tempL.append(itemSet[i])
                itemCount.append(itemCounter)
            else:
                newDiscardedValue.append(itemSet[i])

    return tempL,itemCount,newDiscardedValue
        


In [178]:
receivedL,receiveditemCount,receivedDiscarededValue  =  get_frequent(c[initialSize],Transactions,minimumSupport,discarded_transactions)
l.update({initialSize:receivedL})
supportCount.update({initialSize:receiveditemCount})
discarded_transactions.update({initialSize:receivedDiscarededValue})

In [179]:
print(receivedL,'candidates above threshold ')
print(receiveditemCount)
print(receivedDiscarededValue)
print(c,'amount of candidates')

[['0'], ['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9']] candidates above threshold 
[735, 838, 827, 852, 858, 800, 815, 808, 833, 813]
[]
{1: [['0'], ['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9']]} amount of candidates


In [180]:
def join_two_itemsets(firstItem,secondItem,sortedOrder):
    firstItem.sort(key=lambda y: sortedOrder.index(y))
    secondItem.sort(key=lambda j: sortedOrder.index(j))

    for i in range(len(firstItem)):
        if firstItem[i] == secondItem[i]:
            return []
    if sortedOrder.index(firstItem[-1]) < sortedOrder.index(secondItem[-1]):
            return firstItem + [secondItem[-1]]

    return []


In [181]:
def joinItemset(items,initialItem):
    # print(items,initialItem,"i am joinItemset")
    tempC = []
    for i in range(len(items)-1):
        for j in range(i+1,len(items)):
            # print(items[i],items[j],initialItem,"i am joinItemset sending this")

            itOut = join_two_itemsets(items[i],items[j],initialItem)
            if len (itOut)>0:
                tempC.append(itOut)
    return tempC
            

In [ ]:
print()

In [182]:
totalSize = initialSize + totalSize
noMoreItems = False
while noMoreItems == False:
    joinedSet = joinItemset(l[totalSize-1],sortedOrder)
    # joinedSet = joinItemset(l[totalSize],sortedOrder)
    # print(joinedSet)
    c.update({totalSize:joinedSet})
    receivedL,receiveditemCount,receivedDiscarededValue = get_frequent(c[totalSize],Transactions,minimumSupport,discarded_transactions)
    discarded_transactions.update({totalSize:receivedDiscarededValue})
    l.update({totalSize:receivedL})
    supportCount.update({totalSize:receiveditemCount})
    totalSize = totalSize+initialSize
    if len(receivedL) <1:
        noMoreItems = True
    noMoreItems = True

In [183]:
print(l)
print(discarded_transactions)

{1: [['0'], ['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9']], 2: [['0', '1'], ['0', '2'], ['0', '3'], ['0', '4'], ['0', '5'], ['0', '6'], ['0', '7'], ['0', '8'], ['0', '9'], ['1', '2'], ['1', '3'], ['1', '4'], ['1', '5'], ['1', '6'], ['1', '7'], ['1', '8'], ['1', '9'], ['2', '3'], ['2', '4'], ['2', '5'], ['2', '6'], ['2', '7'], ['2', '8'], ['2', '9'], ['3', '4'], ['3', '5'], ['3', '6'], ['3', '7'], ['3', '8'], ['3', '9'], ['4', '5'], ['4', '6'], ['4', '7'], ['4', '8'], ['4', '9'], ['5', '6'], ['5', '7'], ['5', '8'], ['5', '9'], ['6', '7'], ['6', '8'], ['6', '9'], ['7', '8'], ['7', '9'], ['8', '9']]}
{1: [], 2: []}
